In [ ]:
!pip install -q transformers gradio newspaper3k sentencepiece lxml_html_clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from newspaper import Article
import gradio as gr

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
bias_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu
Device set to use cpu


In [ ]:
bias_labels = ["left-wing", "right-wing", "center", "neutral", "liberal", "conservative"]

In [ ]:
def analyze_news(url):
    # Step 1: Extract article
    article = Article(url)
    article.download()
    article.parse()
    full_text = article.text

    # Step 2: Generate Summary (used as headline)
    summary = summarizer(full_text, max_length=60, min_length=15, do_sample=False)[0]['summary_text']

    # Step 3: Bias Detection
    bias_result = bias_classifier(full_text, bias_labels)
    top_bias = bias_result["labels"][0]
    confidence = bias_result["scores"][0]

    return article.title, summary, top_bias, round(confidence * 100, 2)


In [ ]:
interface = gr.Interface(
    fn=analyze_news,
    inputs=gr.Textbox(label="Enter News Article URL"),
    outputs=[
        gr.Textbox(label="Original Title"),
        gr.Textbox(label="Generated Headline (Summary)"),
        gr.Textbox(label="Detected Bias"),
        gr.Textbox(label="Bias Confidence (%)")
    ],
    title="📰 News Analyzer",
    description="Enter a news article URL. This app summarizes it and detects political bias.",
)

In [ ]:
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1cf3bf870ba234ed7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
